# Import Required Libraries
Import necessary libraries such as PyTorch, torchvision, and any other dependencies.

In [ ]:
# Import necessary libraries
import torch
import torchvision
from torch import nn, optim
from torchvision import datasets, transforms, models

# Additional dependencies
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Load the Mapillary Dataset
Load the Mapillary dataset using torchvision or a custom data loader.

In [ ]:
# Load the Mapillary Dataset

# Define the transformation for the dataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])

# Load the Mapillary dataset
# Assuming the dataset is in a directory named 'mapillary_dataset'
dataset_path = 'path/to/mapillary_dataset'
mapillary_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(mapillary_dataset))
val_size = len(mapillary_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(mapillary_dataset, [train_size, val_size])

# Create data loaders for training and validation
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Display the size of the datasets
print(f'Training dataset size: {train_size}')
print(f'Validation dataset size: {val_size}')

# Preprocess the Dataset
Preprocess the dataset, including resizing, normalization, and data augmentation.

In [ ]:
# Data Augmentation
augmentation_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(10),  # Randomly rotate images by 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change the brightness, contrast, saturation, and hue
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])

# Apply data augmentation to the training dataset
train_dataset.dataset.transform = augmentation_transform

# Display a few augmented images
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get a batch of training data
dataiter = iter(train_loader)
images, labels = dataiter.next()

# Show images
imshow(torchvision.utils.make_grid(images))

# Load SAM Checkpoints
Load the pre-trained SAM checkpoints from the downloaded files.

In [ ]:
# Load SAM Checkpoints

# Define the path to the SAM checkpoints
sam_checkpoint_path = 'path/to/sam_checkpoints'

# Load the SAM model
sam_model = torch.load(sam_checkpoint_path)

# Set the model to evaluation mode
sam_model.eval()

# Print the model architecture
print(sam_model)

# Define the Model Architecture
Define the SAM model architecture and modify it if necessary for the specific task.

In [ ]:
# Define the Model Architecture

# Define the SAM model architecture and modify it if necessary for the specific task
class SAMModel(nn.Module):
    def __init__(self, pretrained_model):
        super(SAMModel, self).__init__()
        self.pretrained_model = pretrained_model
        
        # Modify the final layer to match the number of classes in the Mapillary dataset
        num_ftrs = self.pretrained_model.fc.in_features
        self.pretrained_model.fc = nn.Linear(num_ftrs, len(mapillary_dataset.classes))
    
    def forward(self, x):
        return self.pretrained_model(x)

# Initialize the SAM model with the pretrained weights
model = SAMModel(sam_model)

# Print the modified model architecture
print(model)

# Set Up Training Configuration
Set up the training configuration, including loss function, optimizer, and learning rate scheduler.

In [ ]:
# Set Up Training Configuration

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Print the training configuration
print("Loss function:", criterion)
print("Optimizer:", optimizer)
print("Learning rate scheduler:", scheduler)

# Train the Model
Train the SAM model on the preprocessed Mapillary dataset.

In [ ]:
# Train the Model

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Number of epochs to train the model
num_epochs = 25

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = train_loader
        else:
            model.eval()  # Set model to evaluate mode
            dataloader = val_loader
        
        running_loss = 0.0
        running_corrects = 0
        
        # Iterate over data
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                # Backward pass and optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        if phase == 'train':
            scheduler.step()
        
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    
    print()

# Save the trained model
torch.save(model.state_dict(), 'sam_finetuned_mapillary.pth')

# Evaluate the Model Performance
Evaluate the performance of the fine-tuned model on a validation set.

In [ ]:
# Evaluate the Model Performance

# Set the model to evaluation mode
model.eval()

# Initialize variables to track performance
running_loss = 0.0
running_corrects = 0

# Disable gradient computation for evaluation
with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

# Calculate loss and accuracy
val_loss = running_loss / len(val_loader.dataset)
val_acc = running_corrects.double() / len(val_loader.dataset)

print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_acc:.4f}')

# Save the Fine-Tuned Model
Save the fine-tuned model to disk for future use.

In [ ]:
# Save the Fine-Tuned Model

# Save the fine-tuned model to disk for future use
torch.save(model.state_dict(), 'sam_finetuned_mapillary.pth')

# Verify that the model has been saved correctly by loading it back
loaded_model = SAMModel(sam_model)
loaded_model.load_state_dict(torch.load('sam_finetuned_mapillary.pth'))
loaded_model.to(device)

# Print the loaded model to confirm
print("Loaded fine-tuned model:")
print(loaded_model)